# How to Run
## A guide to getting started with WFP
This Jupyter notebook should give a comprehensive overview for figuring out how to run the imaka wavefront profiler. 

## => Required Python Libraries
Below is an import list of all functions used throughout this code. If you're able to run the following block of code, you should be able to run this package without issues.

!TODO: check that all libraries are used throughout code

In [1]:
#Pipeline
import os
import sys
import fnmatch
import logging
import threading
import time
import concurrent.futures
#Correlator
import math
import imageio
import pandas as pd
import numpy as np
from scipy.io import readsav
import matplotlib
from astropy.table import Table
from astropy.stats import sigma_clip
from astropy.io import fits
#Plotting
from celluloid import Camera
from IPython.display import Image 

The following imports are from seperate packages that aren't necessary for core functionality:

In [2]:
#datatable: not required for corr code
from datetime import datetime, date, time, timezone
import pytz

In [3]:
# estimator
import hdbscan
import seaborn as sns

## => Configuration Files
Configuration files consist of the directory locations you wish the correlator function to find and store data. 

### Defining a variable path: 
name of variable a space, an equals, a space, value, line break. This is the order that the file reader will parse inputs.

> name = value

The file reader for this file will not catch anything that starts with "#" so feel free to use those as comments.

### The major variables to edit: 
The big variables to edit are the paths specific to your system. They are described below:

> data_path = /home/imaka/data/

The data path shows the system where to look for files that contain dates. for example, this is the directory where we'd expect to find **/home/imaka/data/**20201102

> out_path = /home/emcewen/out/

This is where you'll be storing your data output. The system will create this path if it can, so be sure you're pointing it in the right direction

> target_path = /home/emcewen/data/target_input/

The target path shows the system where to look for target files. In this package, target files are listed under: **wf_profile_inputs_targets**


### Varibles that don't need changing:
These are ways to customize how the pipeline operates or what it outputs. Don't feel the need to change them if you don't need to. 

> pipe_fn = all

This determines whether you want just data fits, plots from previous files, or both at once. **all** should work for most users.

> parallel = True

This allows you to do a parallization of the conf run, but is experimental.

In [8]:
# print an example conf file

f = open('inputs/conf/CONF_data.txt', 'r')
print (f.read())

# Configure file for DATA
# => Change data paths here

# Commented lines will be ignored
# variables must be defined with " = " (spaces included)

# where to place files
data_path = /home/imaka/data/
out_path = /home/emcewen/out/
target_path = /home/emcewen/data/target_input/

# options: all, data, plots
pipe_fn = all

# Parallel processing
parallel = True


## => Run files

Run files are inputs to the main pipeline that outlines either the dates or the files for the correlator to run over. 

Entries can be:
- **a date**, will be looked for in **data_path** and all fits files in date/ or date/ao will be added to a list of files to process
- **target name**, will look for this file in the **target_path** defined in conf, where the file begins with **target_name** and ends with **_USNOB_GS.txt**. If a new target is inserted, all dates after the change will be given that target until switched again. 


In [7]:
# Example Run File: Data

f = open('inputs/runs/RUN10.txt', 'r')
print (f.read())

Beehive-W
20190226


In [ ]:
# Example Run File: Simulation

## => Target files

Target files are optional, but will give more information to 
